In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
from torch import Tensor
from typing import Type
from torchvision.transforms.autoaugment import AutoAugmentPolicy

In [2]:
# Data transforms (normalization & data augmentation)
# stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# train_tfms = transforms.Compose([transforms.Resize((64, 64)),
#                          transforms.RandomCrop(64, padding=4, padding_mode='reflect'),
#                          transforms.RandomHorizontalFlip(),
#                          transforms.ToTensor(),
#                          transforms.Normalize(*stats,inplace=True)])
# valid_tfms = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.Normalize(*stats)])

In [3]:
train_tfms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for test data
valid_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
traindata = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_tfms)
validationdata = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=valid_tfms)

100%|██████████| 170498071/170498071 [00:13<00:00, 12411633.74it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
len(traindata)

50000

In [6]:
len(validationdata)

10000

In [7]:
image, label = traindata[0]
print(image.shape, label)

torch.Size([3, 32, 32]) 6


In [8]:
trainDataLoder = torch.utils.data.DataLoader(traindata, batch_size=64, num_workers=2, shuffle=True)
validationDataLoder = torch.utils.data.DataLoader(validationdata, batch_size=64, num_workers=2, shuffle=False)

##Custom ResNet

In [9]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool:
      layers.append(nn.MaxPool2d(2))
      #layers.append(nn.Dropout(0.1))
    return nn.Sequential(*layers)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.conv1 = conv_block(in_channels, 64)
        self.res1 = nn.Sequential(conv_block(64, 64), conv_block(64, 64))

        self.conv2 = conv_block(64, 128, pool=True)
        self.res2 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        self.res2_1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool=True)
        self.res3 = nn.Sequential(conv_block(256, 256), conv_block(256, 256))
        self.res3_1 = nn.Sequential(conv_block(256, 256), conv_block(256, 256))

        self.conv4 = conv_block(256, 512, pool=True)

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        #nn.Dropout(0.5),
                                        nn.Linear(512, num_classes)
                                       )

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.res1(out) + out
        out = self.conv2(out)
        out = self.res2(out) + out
        out = self.res2_1(out) + out
        out = self.conv3(out)
        out = self.res3(out) + out
        out = self.res3_1(out) + out
        out = self.conv4(out)
        out = self.classifier(out)
        return out

In [10]:
model = ResNet9(3,10).cuda()


In [11]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

4,585,866 total parameters.
4,585,866 training parameters.


In [12]:
from torchsummary import summary

summary(model,(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,928
       BatchNorm2d-8           [-1, 64, 32, 32]             128
              ReLU-9           [-1, 64, 32, 32]               0
           Conv2d-10          [-1, 128, 32, 32]          73,856
      BatchNorm2d-11          [-1, 128, 32, 32]             256
             ReLU-12          [-1, 128, 32, 32]               0
        MaxPool2d-13          [-1, 128, 16, 16]               0
           Conv2d-14          [-1, 128,

In [13]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay = 1e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [14]:
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [16]:
torch.save(model.state_dict(), 'model.pt')

In [15]:
train_loss_history = []
train_acc_history = []
validation_loss_history = []
validation_acc_history = []

for epoch in range(60):
  train_loss = 0.0
  train_acc = 0.0
  validation_loss = 0.0
  validation_acc = 0.0

  model.train()
  for i, data in enumerate(trainDataLoder):
    images, labels = data
    images = images.cuda()
    labels = labels.cuda()
    optimizer.zero_grad()
    predicted_labels = model(images) ## forward pass
    fit = loss(predicted_labels, labels)  ## calculate loss
    fit.backward() ## backprop

#     nn.utils.clip_grad_value_(model.parameters(), 0.1)

    optimizer.step()  ## calculate gradients and updates weights
    train_loss += fit.item()
    train_acc += calculate_accuracy(predicted_labels, labels)

  model.eval()
  for i, data in enumerate(validationDataLoder):
    with torch.no_grad():
      images, labels = data
      images = images.cuda()
      labels = labels.cuda()
      predicted_labels = model(images)
      fit = loss(predicted_labels, labels)
      validation_loss += fit.item()
      validation_acc += calculate_accuracy(predicted_labels, labels)

  train_loss = train_loss/len(trainDataLoder)
  validation_loss = validation_loss/len(validationDataLoder)
  scheduler.step(validation_loss)
  train_acc = train_acc/len(trainDataLoder)
  validation_acc = validation_acc/len(validationDataLoder)

  train_loss_history.append(train_loss)
  validation_loss_history.append(validation_loss)
  train_acc_history.append(train_acc)
  validation_acc_history.append(validation_acc)

  print(f'Epochs {epoch}:')
  print(f'        Train loss: {train_loss}, Validation loss: {validation_loss}')
  print(f'        Train Acc: {train_acc}, Validation Acc: {validation_acc}')

  if (epoch+1)%20==0:
     torch.save(model.state_dict(), f'model_{epoch+1}.pt')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epochs 0:
        Train loss: 1.3777689230259118, Validation loss: 1.0485081528402438
        Train Acc: 0.4973025918006897, Validation Acc: 0.6429139971733093
Epochs 1:
        Train loss: 0.8409694927503996, Validation loss: 0.811320357451773
        Train Acc: 0.700607419013977, Validation Acc: 0.7297969460487366
Epochs 2:
        Train loss: 0.6564901880062449, Validation loss: 0.5952430821148453
        Train Acc: 0.7732976078987122, Validation Acc: 0.8007563948631287
Epochs 3:
        Train loss: 0.560501814002881, Validation loss: 0.5098637946092399
        Train Acc: 0.8053069114685059, Validation Acc: 0.8303145170211792
Epochs 4:
        Train loss: 0.49199222161641815, Validation loss: 0.4948246654621355
        Train Acc: 0.8290441036224365, Validation Acc: 0.8286226391792297
Epochs 5:
        Train loss: 0.4343110920332582, Validation loss: 0.4459282039267242
        Train Acc: 0.8495444655418396, Validation Acc: 0.8492237329483032
Epochs 6:
        Train loss: 0.3926307091

In [17]:
from google.colab import files
uploaded = files.upload()

Saving cifar_test_nolabels.pkl to cifar_test_nolabels.pkl


In [18]:
import os
import pickle
import numpy as np

def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

# Load the batch
cifar10_batch = load_cifar_batch('cifar_test_nolabels.pkl')

# Extract images
images = cifar10_batch[b'data'].reshape(-1, 3, 32, 32)
ids = cifar10_batch[b'ids']
pred_labels = np.zeros(ids.shape,"int32")

In [19]:
for i in range(10000):
  model.eval()
  with torch.no_grad():
    test_image = images[i]
    test_image = np.transpose(test_image, (1, 2, 0))
    test_image = valid_tfms(test_image)
    test_image = np.transpose(test_image, (0, 1, 2))
    test_image = test_image.cuda()
    test_image = test_image.unsqueeze(0)
    pred = model(test_image)
    _, predicted = torch.max(pred, 1)
    pred_labels[i] = predicted

import pandas as pd
test_results = pd.DataFrame({'ID': ids, 'Labels': pred_labels})
test_results.to_csv('result.csv', index=False)